In [674]:
from nltk.corpus import reuters
 
from nltk import word_tokenize
from nltk.corpus import reuters 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

cachedStopWords = stopwords.words("english")

def tokenize(text):
    min_length = 3
    words = map(lambda word: word.lower(), word_tokenize(text));
    words = [word for word in words if word not in cachedStopWords]
    tokens =(list(map(lambda token: PorterStemmer().stem(token), words)));
    p = re.compile('[a-zA-Z]+');
    filtered_tokens = list(filter(lambda token: p.match(token) and len(token)>=min_length, tokens));
    return filtered_tokens

# Return the representer, without transforming
def tf_idf(docs):	
    tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=3, max_df=0.90, max_features=1000, use_idf=True, sublinear_tf=True);
    tfidf.fit(docs);
    return tfidf;

def feature_values(doc, representer):
    doc_representation = representer.transform([doc])
    features = representer.get_feature_names()
    return [(features[index], doc_representation[0, index]) for index in doc_representation.nonzero()[1]]

def collection_stats():
    # List of documents
    documents = reuters.fileids()
    print(str(len(documents)) + " documents");
    
    train_docs = list(filter(lambda doc: doc.startswith("train"), documents));
    print(str(len(train_docs)) + " total train documents");
    
    test_docs = list(filter(lambda doc: doc.startswith("test"), documents));	
    print(str(len(test_docs)) + " total test documents");

    # List of categories 
    categories = reuters.categories();
    print(str(len(categories)) + " categories");

    # Documents in a category
    category_docs = reuters.fileids("acq");

    # Words for a document
    document_id = category_docs[0]
    document_words = reuters.words(category_docs[0]);
    print(document_words);

    # Raw document
    print(reuters.raw(document_id));

def main():
    train_docs = []
    test_docs  = []

    for doc_id in reuters.fileids():
        if doc_id.startswith("train"):
            train_docs.append(reuters.raw(doc_id))
        else:
            test_docs.append(reuters.raw(doc_id))
        
    representer = tf_idf(train_docs);

    for doc in test_docs:
        print(feature_values(doc, representer))

In [1854]:
import numpy as np
import lda
import lda.datasets

X = lda.datasets.load_reuters()
X.shape

(395, 4258)

In [2036]:
import numpy as np
import pandas as pd
Wi           = []
documents    = []
train_doc_id = []
doc_indice   = 0
totallen     = len(reuters.fileids())
desired_categ= [1,3,6,10,19]
myfileid    = []
myfilecateg = []
doc_names    = []
for i in desired_categ:
    for docid in reuters.fileids(reuters.categories()[i]):
        if docid.startswith("train"):
            myfileid.append(docid)
            myfilecateg.append(reuters.categories()[i])
myfileids = set(myfileid)

new_categs = []
for doc_id in myfileids:
    if doc_id.startswith("train"):
        tokenized_doc = tokenize(reuters.raw(doc_id))
        if len(tokenized_doc)>0:
            new_categs.append(myfilecateg[myfileid.index(doc_id)])
            train_doc_id.append(doc_id)
            doc_indice = doc_indice + 1
            Wi = Wi + tokenized_doc
            doclist = len(tokenized_doc)*[doc_indice]
            documents = documents + doclist
            doc_names = doc_names + len(tokenized_doc)*[doc_id]
        
d = {'corpus_words':Wi,'corpus_documents':documents,'doc_ids':doc_names}
corpus_df = pd.DataFrame(d)

In [1968]:
i = 1
print(reuters.categories()[i])
#reuters.fileids(reuters.categories()[i])

alum


In [1876]:
#reuters.raw('training/8933',)

In [2037]:
# Dirichlet priors
alpha      = 1
beta       = 1
vocabulary = set(Wi)
vocabulary = list(vocabulary)

#set the numbers
n_docs = doc_indice
n_topics = 5
vocab_size = len(vocabulary)

#get matrix
matrix = np.zeros((n_docs, vocab_size))
doc_list = []
for m in range(n_docs):
    doc_list.append([train_doc_id[m]] * corpus_df.corpus_documents.value_counts()[m+1])
    df_doc = corpus_df.loc[corpus_df['corpus_documents'] == (m+1)]
    doc_unique_words = df_doc.corpus_words.unique()
    for docword in doc_unique_words:
        n = vocabulary.index(docword)
        matrix[m,n] = df_doc.corpus_words.value_counts()[docword]
    

In [1973]:
def word_indices(vec):
    for idx in vec.nonzero()[0]:
        for i in range(int(vec[idx])):
            yield idx

In [2038]:
#Initialize
# number of times document m and topic z co-occur
nmz = np.zeros((n_docs, n_topics))
# number of times topic z and word w co-occur
nzw = np.zeros((n_topics, vocab_size))
nm = np.zeros(n_docs)
nz = np.zeros(n_topics)
topics = {}
for m in range(n_docs):
    # i is a number between 0 and doc_length-1
    # w is a number between 0 and vocab_size-1
    for i, w in enumerate(word_indices(matrix[m, :])):
        # choose an arbitrary topic as first topic for word i
        z = np.random.randint(n_topics)
        nmz[m,z] += 1
        nm[m] += 1
        nzw[z,w] += 1
        nz[z] += 1
        topics[(m,i)] = z

In [2005]:
#a["doc_labels"].values.tolist()
a = np.unique(compare_df["doc_labels"].values)
a.tolist()

'cocoa'

In [2035]:
a = compare_df.loc[compare_df['doc_labels'] == 'dlr']
len(a["doc_lda"].values)

128

In [2041]:
k = {'doc_labels':new_categs,'doc_lda':doctopic}
compare_df = pd.DataFrame(k)
a = compare_df.loc[compare_df['doc_labels'] == 'bop']
a1_lab = a["doc_labels"].values[:15].tolist()
a1_top = a["doc_lda"].values[:15].tolist()
a = compare_df.loc[compare_df['doc_labels'] == 'cocoa']
a2_lab = a["doc_labels"].values[:15].tolist()
a2_top = a["doc_lda"].values[:15].tolist()
a = compare_df.loc[compare_df['doc_labels'] == 'copper']
a3_lab = a["doc_labels"].values[:15].tolist()
a3_top = a["doc_lda"].values[:15].tolist()
a = compare_df.loc[compare_df['doc_labels'] == 'dlr']
a4_lab = a["doc_labels"].values[:15].tolist()
a4_top = a["doc_lda"].values[:15].tolist()
a = compare_df.loc[compare_df['doc_labels'] == 'alum']
a5_lab = a["doc_labels"].values[:15].tolist()
a5_top = a["doc_lda"].values[:15].tolist()
k = {'reuters_bop':a1_lab,'lda_bop':a1_top,'reuters_cocoa':a2_lab,'lda_cocoa':a2_top,'reuters_copper':a3_lab,'lda_copper':a3_top,'reuters_dlr':a4_lab,'lda_dlr':a4_top,'reuters_alum':a5_lab,'lda_alum':a5_top}
dff = pd.DataFrame(k)
dff

,reuters_bop,lda_bop,reuters_cocoa,lda_cocoa,reuters_copper,lda_copper,reuters_dlr,lda_dlr,reuters_alum,lda_alum
0,bop,1.0,cocoa,4.0,copper,3.0,dlr,2.0,alum,3.0
1,bop,1.0,cocoa,4.0,copper,3.0,dlr,0.0,alum,1.0
2,bop,2.0,cocoa,4.0,copper,3.0,dlr,0.0,alum,3.0
3,bop,1.0,cocoa,4.0,copper,3.0,dlr,0.0,alum,3.0
4,bop,1.0,cocoa,4.0,copper,3.0,dlr,0.0,alum,3.0
5,bop,1.0,cocoa,4.0,copper,3.0,dlr,0.0,alum,3.0
6,bop,1.0,cocoa,4.0,copper,3.0,dlr,0.0,alum,3.0
7,bop,1.0,cocoa,4.0,copper,3.0,dlr,0.0,alum,3.0
8,bop,1.0,cocoa,4.0,copper,3.0,dlr,0.0,alum,3.0
9,bop,1.0,cocoa,2.0,copper,3.0,dlr,0.0,alum,3.0


In [2039]:
#Gibbs steps
maxiter = 2000
for it in range(maxiter):
    if it%10==0:
        print(it)
    for m in range(n_docs):
        for i, w in enumerate(word_indices(matrix[m, :])):
            z = topics[(m,i)]
            nmz[m,z] -= 1
            nm[m] -= 1
            nzw[z,w] -= 1
            nz[z] -= 1
            left = (nzw[:,w] + beta) / (nz + beta * vocab_size)
            right = (nmz[m,:] + alpha) / (nm[m] + alpha * n_topics)
            p_z = left * right
            # normalize to obtain probabilities
            p_z /= np.sum(p_z)
            z = np.random.multinomial(1,p_z).argmax()

            nmz[m,z] += 1
            nm[m] += 1
            nzw[z,w] += 1
            nz[z] += 1
            topics[(m,i)] = z

0
10
20
30
40
50
60
70
80
90
100
110
120


KeyboardInterrupt: 

In [2040]:
doctopic = np.zeros(n_docs);
topic_ratio = []
for m in range(n_docs):
    topicss = []
    for i, w in enumerate(word_indices(matrix[m, :])):
        topicss.append(topics[(m,i)])
    doctopic[m] = max(set(topicss), key=topicss.count)
    topic_ratio.append(topicss.count(doctopic[m]) / len(topicss))